In [ ]:
import warnings

import gymnasium as gym
import torch
import torch.nn as nn
from skrl.agents.torch.ppo import PPO, PPO_DEFAULT_CONFIG
from skrl.envs.wrappers.torch import wrap_env
from skrl.memories.torch import RandomMemory
from skrl.models.torch import DeterministicMixin, Model, MultivariateGaussianMixin
from skrl.resources.schedulers.torch import KLAdaptiveLR
from skrl.trainers.torch import ParallelTrainer
from skrl.utils import set_seed
from skrl.utils.spaces.torch import unflatten_tensorized_space
from torch.nn import TransformerEncoder, TransformerEncoderLayer

from preprocess import preprocess

set_seed(42)

gym.register(
    id="MultiDatasetDiscretedTradingEnv",
    entry_point="predict_next_candle:MultiDatasetDiscretedTradingEnv",
    disable_env_checker=True,
)

In [ ]:
env_cfg = dict(
    id="MultiDatasetDiscretedTradingEnv",
    dataset_dir="./data/futures/15m/**/**/*.pkl",
    preprocess=preprocess,
    max_episode_duration=2000,
    verbose=0,
    window_size=30,
    btc_index=True,
)

In [ ]:
env = gym.make(**env_cfg)
obs = env.observation_space
env = gym.make_vec(
    vectorization_mode="async",
    num_envs=512,
    **env_cfg,
)
env = wrap_env(env, wrapper="gymnasium")

In [ ]:
device = env.device
replay_buffer_size = 1024 * 1 * env.num_envs
memory_size = int(replay_buffer_size / env.num_envs)
memory = RandomMemory(memory_size=memory_size, num_envs=env.num_envs, device=device, replacement=False)

In [ ]:
class LearnablePositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1024 * 16):
        super().__init__()
        self.position_embedding = nn.Embedding(max_len, d_model)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)
        positions = positions.expand(batch_size, seq_len)
        position_encoded = self.position_embedding(positions)
        return x + position_encoded


class SharedNoFC(MultivariateGaussianMixin, DeterministicMixin, Model):
    def __init__(
        self,
        observation_space,
        action_space,
        device,
        clip_actions=False,
        clip_log_std=True,
        min_log_std=-20,
        max_log_std=2,
    ):
        Model.__init__(self, observation_space, action_space, device)
        MultivariateGaussianMixin.__init__(
            self, clip_actions, clip_log_std, min_log_std, max_log_std
        )
        DeterministicMixin.__init__(self, clip_actions)

        self._shared_features = None
        self.num_features = 7
        self.net_projection = nn.Sequential(
            nn.Conv1d(self.num_features, 8, kernel_size=1, padding=1),
        )

        # Transformer Encoder for self-attention
        transformer_layer = TransformerEncoderLayer(
            d_model=8,  # The size of the input feature vector
            nhead=4,  # Number of attention heads
            dim_feedforward=256,  # The size of the feedforward network in the encoder
            dropout=0.1,
            batch_first=True,  # Use batch_first for better inference performance
        )
        self.positional_encoding = LearnablePositionalEncoding(d_model=8)
        self.transformer_encoder = TransformerEncoder(transformer_layer, num_layers=2)

        self.shared_head = nn.Sequential(
            nn.Conv1d(8, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=1),
            nn.Flatten(),
        )

        self.policy_head = nn.Sequential(
            # nn.Linear(self.num_actions * 32, 32),
            # nn.ReLU(),
            nn.Linear(2048, self.num_actions),
        )
        self.value_head = nn.Sequential(
            # nn.Linear(self.num_actions * 32, 32),
            # nn.ReLU(),
            nn.Linear(2048, 1),
        )
        self.log_std_parameter = nn.Parameter(torch.zeros(self.num_actions))

    def act(self, inputs, role):
        if role == "policy":
            return MultivariateGaussianMixin.act(self, inputs, role)
        elif role == "value":
            return DeterministicMixin.act(self, inputs, role)

    def compute(self, inputs, role):
        states = unflatten_tensorized_space(obs, inputs["states"])

        if role == "policy":
            features = states.permute(0, 2, 1)
            features = self.net_projection(features)
            features = self.positional_encoding(features.permute(0, 2, 1))
            features = features + self.transformer_encoder(features)

            self._shared_features = self.shared_head(features.permute(0, 2, 1))
            actions = self.policy_head(self._shared_features)
            return actions, self.log_std_parameter, {}

        elif role == "value":
            if self._shared_features is None:
                features = states.permute(0, 2, 1)
                features = self.net_projection(features)
                features = self.positional_encoding(features.permute(0, 2, 1))
                features = features + self.transformer_encoder(features)

                shared_features = self.shared_head(features.permute(0, 2, 1))
            else:
                shared_features = self._shared_features

            self._shared_output = None

            value = self.value_head(shared_features)
            return value, {}

In [ ]:
class LearnablePositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1024 * 16):
        super().__init__()
        self.position_embedding = nn.Embedding(max_len, d_model)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)
        positions = positions.expand(batch_size, seq_len)
        position_encoded = self.position_embedding(positions)
        return x + position_encoded


class AttentionPooling(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionPooling, self).__init__()
        self.attention = nn.Linear(hidden_size, 1)

    def forward(self, x):
        attention_weights = self.attention(x).squeeze(-1)
        attention_weights = torch.softmax(attention_weights, dim=-1)
        pooled_x = torch.bmm(attention_weights.unsqueeze(1), x).squeeze(1)
        return pooled_x


class SharedNoFC(MultivariateGaussianMixin, DeterministicMixin, Model):
    def __init__(
        self,
        observation_space,
        action_space,
        device,
        clip_actions=False,
        clip_log_std=True,
        min_log_std=-20,
        max_log_std=2,
    ):
        Model.__init__(self, observation_space, action_space, device)
        MultivariateGaussianMixin.__init__(
            self, clip_actions, clip_log_std, min_log_std, max_log_std
        )
        DeterministicMixin.__init__(self, clip_actions)

        self._shared_features = None
        self.num_features = 8

        # Transformer Encoder for self-attention
        transformer_layer = TransformerEncoderLayer(
            d_model=self.num_features,  # The size of the input feature vector
            nhead=4,  # Number of attention heads
            dim_feedforward=256,  # The size of the feedforward network in the encoder
            dropout=0.1,
            batch_first=True,  # Use batch_first for better inference performance
        )
        self.positional_encoding = LearnablePositionalEncoding(
            d_model=self.num_features
        )
        self.transformer_encoder = TransformerEncoder(transformer_layer, num_layers=2)
        self.attention_pooling = AttentionPooling(self.num_features)

        self.shared_head = nn.Sequential(
            nn.LayerNorm(8),
            nn.GELU(),
            nn.Dropout(0.3),
        )

        self.policy_head = nn.Sequential(
            nn.Linear(self.num_features, self.num_actions),
            # nn.Softplus(),
        )
        self.value_head = nn.Sequential(
            nn.Linear(self.num_features, 1),
            # nn.Softplus(),
        )

        self.log_std_parameter = nn.Parameter(torch.zeros(self.num_actions))

    def act(self, inputs, role):
        if role == "policy":
            return MultivariateGaussianMixin.act(self, inputs, role)
        elif role == "value":
            return DeterministicMixin.act(self, inputs, role)

    def compute(self, inputs, role):
        states = unflatten_tensorized_space(
            obs, inputs["states"]
        )  # (batch_size, seq_length=30, num_features=8)

        if role == "policy":
            features = self.positional_encoding(states)
            features = self.transformer_encoder(features)
            features = self.attention_pooling(features)
            self._shared_features = self.shared_head(features)

            actions = self.policy_head(self._shared_features)
            return actions, self.log_std_parameter, {}

        elif role == "value":
            if self._shared_features is None:
                features = self.positional_encoding(states)
                features = self.transformer_encoder(features)
                features = self.attention_pooling(features)
                shared_features = self.shared_head(features)
            else:
                shared_features = self._shared_features

            self._shared_output = None

            value = self.value_head(shared_features)
            return value, {}

In [ ]:
models = {}
models["policy"] = SharedNoFC(env.observation_space, env.action_space, device)
models["value"] = models["policy"]

for model in models.values():
    model.init_parameters(method_name="normal_", mean=0.0, std=0.1)

In [ ]:
cfg = PPO_DEFAULT_CONFIG.copy()
cfg["rollouts"] = memory_size
cfg["learning_epochs"] = 16
cfg["mini_batches"] = 12
cfg["discount_factor"] = 0.99
cfg["learning_rate"] = 5e-4
cfg["learning_rate_scheduler"] = KLAdaptiveLR
cfg["learning_rate_scheduler_kwargs"] = {"kl_threshold": 0.01, "min_lr": 1e-8}

cfg["experiment"]["write_interval"] = 1000
cfg["experiment"]["checkpoint_interval"] = 10000
cfg["experiment"]["directory"] = "runs/torch/mddt"

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

agent = PPO(
    models=models,
    memory=memory,
    cfg=cfg,
    observation_space=env.observation_space,
    action_space=env.action_space,
    device=device,
)
path = "24-12-13_09-43-09-912798_PPO"
# path = "24-12-11_10-57-22-046603_PPO"
# agent.load(f"/home/pitin/Desktop/hp/runs/torch/mddt/{path}/checkpoints/best_agent.pt")
cfg_trainer = {"timesteps": 10000000, "headless": True, "environment_info": ["reward"]}
trainer = ParallelTrainer(cfg=cfg_trainer, env=env, agents=[agent])

In [ ]:
trainer.train()